# Set environment variables

In [1]:
import os

os.environ["path"] = r'C:\Users\Admin\Documents\Github\PyDev-Operational-Library-For-Data-Engineers'


os.environ["records_folder"] = r'C:\Users\Admin\Documents\Sketch ai'
os.environ["df_file_name"] = 'export-companies.csv'

os.environ["account_id"] = '20025332'
os.environ["add_hubspot_records_view"] = 'Yes'

# It is not necessary to fill in all of the following fields
os.environ["company_name_column_name"] = 'Company name'
os.environ["phone_number_column_name"] = 'Phone Number' # could be empty ''
os.environ["street_address_column_name"] = 'Street Address' # could be empty ''
os.environ["list_of_columns"] = ''#'Company Domain Name' # could be empty '' or several properties separed with a ; 'Company Domain;Phone Number'

# Libraries

### Python Libraries

In [2]:
import pandas as pd
import numpy as np

### Data Team Libraries

Add your "Operational-Library-For-Data-Engineers" Repository Path _**(path: str)**_ folder where you have saved your repository copy.

In [3]:
path = os.getenv('path')

In [4]:
import sys
sys.path.insert(0,path)

In [5]:
from functions.manage_duplicates.find_duplicate_companies_by_specified_fields import find_duplicate_companies_by_specified_fields
from functions.data_transformations.delete_unnecessary_blank_spaces import delete_unnecessary_blank_spaces
from functions.manage_duplicates.standardize_data.standarize_company_name import standarize_company_name

# Proyect Folders

**Records Folder** _**(records_folder: str)**_ Folder where data is saved.

In [6]:
records_folder = os.getenv('records_folder')

# Read DataFrame

**DataFrame File Name** _**(df_file_name: str)**_ Name of the file to read, should be a csv file type.

In [7]:
df_file_name = os.getenv('df_file_name')

**Read your csv file**

In [8]:
df = pd.read_csv(records_folder+'\\'+df_file_name, low_memory=False)

In [9]:
df.head(2)

,Record ID,Company name,Company Domain Name,Phone Number,Street Address
0,27927988238,Black Iron Metalwork,blackironmetalwork.com,+1 405-760-5190,1901 Northeast 10th Street
1,27921112398,markandson.net,markandson.net,NaN,NaN


**Standarize empty spaces**

In [10]:
df = df.replace(np.nan, '')

**Space to review keys**

In [11]:
#list(df.keys())

# Find Duplicates

In [12]:
company_name_column_name = os.getenv('company_name_column_name')
phone_number_column_name = os.getenv('phone_number_column_name')
street_address_column_name = os.getenv('street_address_column_name')

def setup_lists_of_columns():
    list_of_columns = os.getenv('list_of_columns')
    list_of_columns = list_of_columns.split(';')
    if '' in list_of_columns:
        list_of_columns.remove('')
    return list_of_columns

In [13]:
setup_lists_of_columns()

[]

In [14]:
street,no_duplicates_frame = find_duplicate_companies_by_specified_fields(frame_data = df, 
                                                                                     company_name_column_name = company_name_column_name,
                                                                                     phone_number_column_name = '',
                                                                                     street_address_column_name = street_address_column_name,
                                                                                     list_of_columns = setup_lists_of_columns())

In [15]:
phone, no_duplicates_frame = find_duplicate_companies_by_specified_fields(frame_data = df, 
                                                                                     company_name_column_name = company_name_column_name,
                                                                                     phone_number_column_name = phone_number_column_name,
                                                                                     street_address_column_name = '',
                                                                                     list_of_columns = setup_lists_of_columns())

In [16]:
both, no_duplicates_frame = find_duplicate_companies_by_specified_fields(frame_data = df, 
                                                                                     company_name_column_name = company_name_column_name,
                                                                                     phone_number_column_name = phone_number_column_name,
                                                                                     street_address_column_name = street_address_column_name,
                                                                                     list_of_columns = setup_lists_of_columns())

# Add HubSpot Records View and Save Duplicates Frame

In [17]:
def link_to_view_record(record_id):
    
    account_id = os.getenv('account_id') ## Change this according your account
    record_id = str(int(record_id))
    
    record_view = 'https://app.hubspot.com/contacts/{}/record/0-2/{}'.format(account_id, record_id)
    
    return record_view

In [18]:
add_hubspot_records_view = os.getenv('add_hubspot_records_view')

In [19]:
names = ['Street Address','Phone Number', 'Street Address and Phone Number']

In [20]:
for dataframe, name in zip([street, phone, both],names):
    if add_hubspot_records_view == 'Yes':
        dataframe['Record View'] = dataframe['Record ID'].apply(link_to_view_record)
    else:
        pass

    dataframe = dataframe[['Record ID', 'Company name', 'Company Domain Name', 'Phone Number', 'Street Address', 'Record View', 'Key']]

    dataframe.to_excel(records_folder+r'\Duplicate Company Records Found by {}.xlsx'.format(name))

    print(len(dataframe), 'Duplicated Records Found')

12 Duplicated Records Found
0 Duplicated Records Found
0 Duplicated Records Found


# Webiste Urls

### (In theory it's an unique identifier in hubspot but there are ways around this, in some cases companies have the same company domain name but should be treated differently but the client, eg different sites, divisions...)

In [27]:
df[(df.duplicated(subset='Company Domain Name', keep=False))&(df['Company Domain Name']!='')].sort_values('Company Domain Name')[['Record ID', 'Company name', 'Company Domain Name', 'Phone Number', 'Street Address','Company Domain Name']].to_excel(records_folder+'\Companies with duplicate domain names.xlsx',index=False)

In [37]:
df[df['Company name'].apply(standarize_company_name).duplicated(keep=False)][['Record ID', 'Company name', 'Company Domain Name', 'Phone Number', 'Street Address','Company Domain Name']].sort_values('Company name').to_excel(records_folder+'\Companies with duplicate names.xlsx',index=False)

In [44]:
df['Industry'].value_counts()

Industry
Construction                            685
Facilities Services                      19
Airlines/Aviation                        11
Consumer Services                         8
Utilities                                 4
Oil & Energy                              3
Mining & Metals                           3
Mechanical or Industrial Engineering      3
Real Estate                               2
Building Materials                        2
Electrical/Electronic Manufacturing       2
Logistics and Supply Chain                1
Transportation/Trucking/Railroad          1
Consumer Electronics                      1
Architecture & Planning                   1
Security and Investigations               1
Financial Services                        1
Chemicals                                 1
Consumer Goods                            1
Name: count, dtype: int64

In [23]:
print('\n'.join(df.columns))

Record ID
About Us
Annual Revenue
Campaign of last booking in meetings tool
City
Close Date
Company Domain Name
Company Keywords
Company name
Company owner
Country/Region
Country/Region Code
Create Date
Created by user ID
Date of last meeting booked in meetings tool
Days to Close
Description
Employee range
Facebook Company Page
Facebook Fans
First Contact Create Date
First Conversion
First Conversion Date
First Deal Created Date
First Touch Converting Campaign
Google Plus Page
Has been enriched
HubSpot Team
Ideal Customer Profile Tier
Industry
Industry group
Is Public
Last Activity Date
Last Aircall SMS direction
Last Aircall SMS timestamp
Last Booked Meeting Date
Last Contacted
Last Engagement Date
Last Logged Call Date
Last Modified Date
Last Open Task Date
Last Touch Converting Campaign
Latest Traffic Source
Latest Traffic Source Data 1
Latest Traffic Source Data 2
Latest Traffic Source Timestamp
Lead Status
Lifecycle Stage
Likelihood to close
LinkedIn Bio
LinkedIn Company Page
Link